## 模型 3. LSTM 學習三個字符的時間步驟窗口(Three-Char Time Step Window)到一個字符的映射

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.sequence import pad_sequences

model_save_path = '/content/drive/MyDrive/project/LSTM_save_file/LSTM_classifier.hdf5'
tflite_save_path = '/content/drive/MyDrive/project/LSTM_save_file/LSTM_classifier.tflite'

### 合併CSV

In [ ]:
import pandas as pd
import os
Folder_Path = r'/content/drive/MyDrive/project/dataset'          #要拼接的資料夾及其完整路徑，注意不要包含中文
SaveFile_Path =  r'/content/drive/MyDrive/project/datasets_combine'       #拼接後要儲存的檔案路徑
SaveFile_Name = r'combined_data.csv'              #合併後要儲存的檔名

#修改當前工作目錄
# os.chdir(Folder_Path)
#將該資料夾下的所有檔名存入一個列表
file_list = os.listdir()

#讀取第一個CSV檔案幷包含表頭
df = pd.read_csv(Folder_Path +'/'+ file_list[0])   #編碼預設UTF-8，若亂碼自行更改

#將讀取的第一個CSV檔案寫入合併後的檔案儲存
df.to_csv(SaveFile_Path+'/'+ SaveFile_Name,encoding="ascii",index=False)

#迴圈遍歷列表中各個CSV檔名，並追加到合併後的檔案
for i in range(1,len(file_list)):
    df = pd.read_csv(Folder_Path + '/'+ file_list[i])
    df.to_csv(SaveFile_Path+'/'+ SaveFile_Name,encoding="ascii",index=False, header=False, mode='a+', float_format=True)

In [ ]:
# 測試2
import os
import pandas as pd

# 将文件读取出来放一个列表里面 
path = r'/content/drive/MyDrive/project/dataset/point_history_test_01.csv'        #  获取文件目录，下面是所有的表格
 
# 新建列表，存放文件名
file_list = []
 
# 新建列表存放每个文件数据(依次读取多个相同结构的Excel文件并创建DataFrame)
dfs = []
 
for root,dirs,files in os.walk(path): # 第一个为起始路径，第二个为起始路径下的文件夹，第三个是起始路径下的文件。
    for file in files:
        file_path = os.path.join(root, file)
        file_list.append(file_path) # 使用os.path.join(dirpath, name)得到全路径
        df = pd.read_csv(file_path,encoding="gbk") # 将excel转换成DataFrame
        dfs.append(df)   # 多个df的list
# 将多个DataFrame合并为一个
df = pd.concat(dfs)
 
# 写入excel文件，不包含索引数据
df.to_excel(r'E:\dosn2\qiyexinxi\result.xls', index=False)
————————————————
版权声明：本文为CSDN博主「CJ-404」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/qq_37273544/article/details/100192208

### STEP1. 準備訓練用資料

In [ ]:
dataset = '/content/drive/MyDrive/project/datasets_combine/combined_data.csv'
dataset_01 = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))
dataset_length = 5
dataset_01_list = dataset_01.tolist()
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=0)

### STEP2. 資料預處理

In [ ]:
# 重塑 X 資料的維度成為 (samples, time_steps, features)

X = np.reshape(dataset_01_list, (-1, dataset_length, 42))  # <-- 特別注意這裡


# 使用one hot encode 對Y值進行編碼
def preprocess_ylabel(ylabel, time_steps=10):
  y_data=[]
  time_steps = time_steps
  total_times = int(len(ylabel)/time_steps)
  # print(total_times)
  step = 0
  for i in range(total_times):
    a= ylabel[0+ time_steps*step: time_steps+ time_steps*step ]
    y_data.append(a[0])
    step += 1
  return y_data
y = preprocess_ylabel(y_dataset, 5)
y = np.array(y)
y = y.reshape(-1, 1)
y = utils.to_categorical(y)

In [ ]:
#建立Training dataset＆Testing dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    #分20%當成testing dataset
    test_size=0.2,
)

In [ ]:
print(X_train.shape)
print(X_test.shape)

(169, 5, 42)
(43, 5, 42)


### STEP3. 建立模型

In [ ]:
# 創建模型
model = Sequential()
model.add(LSTM(32, input_shape=(X.shape[1], X.shape[2]))) # <-- 特別注意這裡
model.add(Dense(y.shape[1], activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                9600      
_________________________________________________________________
dense (Dense)                (None, 2)                 66        
Total params: 9,666
Trainable params: 9,666
Non-trainable params: 0
_________________________________________________________________


### STEP4. 定義訓練並進行訓練

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(
    x=X_train,
    y=y_train,
    epochs=500,
    verbose=2,
    batch_size=1,
    # validation_split=0.1
)
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'test_loss={test_loss}')
print(f'test_accuracy={test_accuracy}')

Epoch 1/500
169/169 - 3s - loss: 0.6764 - accuracy: 0.5148
Epoch 2/500
169/169 - 0s - loss: 0.5670 - accuracy: 0.7278
Epoch 3/500
169/169 - 0s - loss: 0.4114 - accuracy: 0.8757
Epoch 4/500
169/169 - 0s - loss: 0.3165 - accuracy: 0.8817
Epoch 5/500
169/169 - 0s - loss: 0.3208 - accuracy: 0.8698
Epoch 6/500
169/169 - 0s - loss: 0.2541 - accuracy: 0.8994
Epoch 7/500
169/169 - 0s - loss: 0.2363 - accuracy: 0.9112
Epoch 8/500
169/169 - 0s - loss: 0.1836 - accuracy: 0.9231
Epoch 9/500
169/169 - 0s - loss: 0.1824 - accuracy: 0.9408
Epoch 10/500
169/169 - 0s - loss: 0.1075 - accuracy: 0.9408
Epoch 11/500
169/169 - 0s - loss: 0.1855 - accuracy: 0.9231
Epoch 12/500
169/169 - 0s - loss: 0.1139 - accuracy: 0.9586
Epoch 13/500
169/169 - 0s - loss: 0.1035 - accuracy: 0.9645
Epoch 14/500
169/169 - 0s - loss: 0.1830 - accuracy: 0.9408
Epoch 15/500
169/169 - 0s - loss: 0.0708 - accuracy: 0.9763
Epoch 16/500
169/169 - 0s - loss: 0.0645 - accuracy: 0.9763
Epoch 17/500
169/169 - 0s - loss: 0.0643 - accura

### STEP5. 存檔

In [ ]:


# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)

# Transform model (quantization)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

INFO:tensorflow:Assets written to: /tmp/tmpn5k8ugez/assets


INFO:tensorflow:Assets written to: /tmp/tmpn5k8ugez/assets


21680

### STEP6. 預測結果

In [ ]:
# 讓我們擷取3個字符轉成張量結構 shape:(1,3,1)來進行infer
for pattern in dataX:
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(alphabet))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    print(seq_in, "->", result)

['A', 'B', 'C'] -> D
['B', 'C', 'D'] -> E
['C', 'D', 'E'] -> F
['D', 'E', 'F'] -> G
['E', 'F', 'G'] -> H
['F', 'G', 'H'] -> I
['G', 'H', 'I'] -> J
['H', 'I', 'J'] -> K
['I', 'J', 'K'] -> L
['J', 'K', 'L'] -> M
['K', 'L', 'M'] -> N
['L', 'M', 'N'] -> O
['M', 'N', 'O'] -> P
['N', 'O', 'P'] -> Q
['O', 'P', 'Q'] -> R
['P', 'Q', 'R'] -> S
['Q', 'R', 'S'] -> T
['R', 'S', 'T'] -> U
['S', 'T', 'U'] -> V
['T', 'U', 'V'] -> W
['U', 'V', 'W'] -> X
['V', 'W', 'X'] -> Y
['W', 'X', 'Y'] -> Z


由"模型#3"的表現來看, 當我們以"時間步驟(timesteps)"的形式給出更多的上下文(context)來訓練LSTM模型時, 這時候循環神經網絡在序列資料的學習的效果就可以發揮出它的效用。

"模型#3"在驗證的結果可達到100%的預測準確度(在這個很簡單的26個字母的順序預測的任務上)!

### 參考:
* Jason Brownlee - "[Understanding Stateful LSTM Recurrent Neural Networks in Python with Keras](https://machinelearningmastery.com/understanding-stateful-lstm-recurrent-neural-networks-python-keras/)"

* Keras官網 - [Recurrent Layer](https://keras.io/layers/recurrent/)

* https://github.com/erhwenkuo